In [111]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

Запрос в базу данных:

select CreationDate, Text from Comments tablesample(1 percent)
where CreationDate >= DATEADD(month, -6, GETDATE())





In [122]:
data = pd.read_csv('QueryResults.csv')
data

,CreationDate,Text
0,2021-09-16 15:32:22,Welcome to Stack Overflow! In order to get the...
1,2021-09-16 15:32:23,Ok so how can I change the format? The dates a...
2,2021-09-16 15:32:28,"Try now mate, pls , i updated answer"
3,2021-09-16 15:32:38,I'm guessing that the HTMLness bit is set if t...
4,2021-09-16 15:32:42,"Thanks for the clarification, John"
...,...,...
26523,2022-03-13 03:23:46,"In your repo, this `func compositionLayerInstr..."
26524,2022-03-13 03:23:50,@QueenoftheNorth The two parameters are indica...
26525,2022-03-13 03:24:13,"The code is buggy. I tried `build_path(""http:/..."
26526,2022-03-13 03:24:18,What about start thinking with the following t...


In [113]:
X = data.to_numpy()[:, 1]
X

array(['Welcome to Stack Overflow! In order to get the best responses to your question please take a tour of the [Help Center](https://stackoverflow.com/help). Specifically the [How to Ask a Good Question](https://stackoverflow.com/help/how-to-ask) and [How to Create A Minimal Reproducible Example](https://stackoverflow.com/help/minimal-reproducible-example) articles will help us understand the problem and give a good answer.',
       'Ok so how can I change the format? The dates are saved in the database as mm-dd-yyyy. How can I change them before giving the values to the page?',
       'Try now mate, pls , i updated answer', ...,
       'The code is buggy. I tried `build_path("http://", "/foo/bar", "index.html")` and it returned `\'http:/foo/bar/index.html\'`.',
       'What about start thinking with the following two lines: `x = [1,2,3,4]` `ratio=[a/b for a,b in zip(x[1:], x)]`',
       'please restrict yourself to one question at a time (unless the questions are closely related) - 

In [114]:
def remaker(sentence):
    simb = [',',"'", '!', '.', ',', '?', '*']
    for x in simb:
        sentence = sentence.replace(x, ' ')
    return sentence.lower()


def trend_words(X, d_size = 100):
    d = {}
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')    # оставляет только слова, без знаков припинания
    #stemmer = WordNetLemmatizer()    # приводит слова к нормальной форме
    for line in X:
        s = [w for w in tokenizer.tokenize(line.lower()) if w not in stop_words]
        for word in s:
            if word in d:
                d[word] += 1
            else:
                d[word] = 1
    word_freq_list = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return np.array(word_freq_list[:d_size])


In [115]:
trend_words(X)

array([['https', '3820'],
       ['code', '3438'],
       ['use', '3116'],
       ['com', '3004'],
       ['answer', '2753'],
       ['question', '2559'],
       ['using', '2230'],
       ['need', '2165'],
       ['like', '2106'],
       ['thanks', '2095'],
       ['would', '2077'],
       ['1', '1879'],
       ['please', '1836'],
       ['data', '1764'],
       ['get', '1763'],
       ['one', '1742'],
       ['want', '1723'],
       ['also', '1723'],
       ['problem', '1696'],
       ['error', '1684'],
       ['see', '1628'],
       ['help', '1602'],
       ['work', '1552'],
       ['file', '1537'],
       ['thank', '1501'],
       ['stackoverflow', '1492'],
       ['0', '1487'],
       ['could', '1434'],
       ['try', '1285'],
       ['add', '1283'],
       ['way', '1235'],
       ['2', '1233'],
       ['function', '1215'],
       ['example', '1209'],
       ['know', '1128'],
       ['value', '1114'],
       ['questions', '1103'],
       ['think', '1087'],
       ['first', '1079'],

In [116]:
from rake_nltk import Rake

In [117]:
def add_to_dict(someting, d):
    if someting:
        if isinstance(someting, list):
            for word in someting:
                if word in d:
                    d[word] += 1
                else:
                    d[word] = 1
        else:
            if someting in d:
                d[someting] += 1
            else:
                d[someting] = 1


def super_trend_words(X, size = 100):
    d_phrases = {}
    d_words = {}
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')    # оставляет только слова, без знаков припинания
    # stemmer = WordNetLemmatizer()    # приводит слова к нормальной форме

    for line in X:
        s = [w for w in tokenizer.tokenize(line.lower()) if w not in stop_words]
        new_line = ' '.join(map(str, s))
        rake = Rake()
        rake.extract_keywords_from_text(new_line)
        key_phrase = rake.get_ranked_phrases()
        if key_phrase:
            key_phrase = key_phrase[:3]
        else:
            key_phrase = ''
        key_words = list(rake.get_word_degrees().keys())[:3]
        add_to_dict(key_words, d_words)
        add_to_dict(key_phrase, d_phrases)
    word_freq_list = sorted(d_words.items(), key=lambda x: x[1], reverse=True)
    phrase_freq_list = sorted(d_phrases.items(), key=lambda x: x[1], reverse=True)
    return np.array(word_freq_list[:size]), np.array(phrase_freq_list[:size])


In [118]:
words, phrases = super_trend_words(X)


In [119]:
words

array([['thanks', '1436'],
       ['answer', '1093'],
       ['thank', '1070'],
       ['please', '947'],
       ['question', '829'],
       ['use', '822'],
       ['yes', '804'],
       ['code', '750'],
       ['https', '623'],
       ['need', '576'],
       ['using', '565'],
       ['try', '506'],
       ['want', '505'],
       ['could', '500'],
       ['tried', '480'],
       ['would', '472'],
       ['see', '464'],
       ['also', '464'],
       ['think', '458'],
       ['like', '424'],
       ['error', '394'],
       ['work', '387'],
       ['problem', '384'],
       ['works', '362'],
       ['get', '344'],
       ['add', '335'],
       ['hi', '330'],
       ['know', '322'],
       ['sure', '316'],
       ['much', '305'],
       ['sorry', '275'],
       ['check', '275'],
       ['one', '275'],
       ['solution', '270'],
       ['added', '261'],
       ['provide', '257'],
       ['mean', '257'],
       ['help', '249'],
       ['worked', '246'],
       ['way', '246'],
       ['righ

In [120]:
phrases

array([['please provide enough code others better understand reproduce problem',
        '90'],
       ['please clarify specific problem provide additional details highlight exactly need currently written hard tell exactly asking',
        '80'],
       ['answer could improved additional supporting information please edit add details citations documentation others confirm answer correct find information write good answers help center help answer',
        '76'],
       ['currently written answer unclear please edit add additional details help others understand addresses question asked find information write good answers help center help answer',
        '52'],
       ['please edit question limit specific problem enough detail identify adequate answer',
        '40'],
       ['thank much', '18'],
       ['please trim code make easier find problem follow guidelines create minimal reproducible example https stackoverflow com help minimal reproducible example',
        '10'],
       ['ques

In [121]:
print(phrases[:5, 0])


['please provide enough code others better understand reproduce problem'
 'please clarify specific problem provide additional details highlight exactly need currently written hard tell exactly asking'
 'answer could improved additional supporting information please edit add details citations documentation others confirm answer correct find information write good answers help center help answer'
 'currently written answer unclear please edit add additional details help others understand addresses question asked find information write good answers help center help answer'
 'please edit question limit specific problem enough detail identify adequate answer']
